# Effect of pre-latent linear layer

This tries to prove experimentally that a further layer added to network with linear ativation and dimension in between inner layers and the latent space is not affecting the discrimination power of the latent space itself.

For theoretical demonstration we cha refer to Restricted Boltzman Machines:
[RBM](https://towardsdatascience.com/deep-learning-meets-physics-restricted-boltzmann-machines-part-i-6df5c4918c15)

In [2]:
import numpy as np
import tensorflow as tf

# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors 

import ipysh
%aimport models.base
import Hunch_utils  as Htls
import Hunch_lsplot as Hplt
import Hunch_tSNEplot as Hsne

%aimport models.AEFIT3


In [3]:
class AEFIT3a(models.AEFIT3.AEFIT3):
    ''' General Autoencoder Fit Model for TF 2.0
    '''
    
    def __init__(self, feature_dim=40, latent_dim=2, dprate = 0., scale=1, activation=tf.nn.relu, beta=1.):
        super(AEFIT3a, self).__init__()
        self.latent_dim = latent_dim
        self.feature_dim = feature_dim
        self.dprate = dprate
        self.scale = scale
        self.activation = activation
        self.set_model()
        self.beta = beta
        print('AEFIT3a ready:')

    def set_model(self, training=True):
        feature_dim = self.feature_dim
        latent_dim = self.latent_dim
        if training: dprate = self.dprate
        else: dprate = 0.
        scale = self.scale
        activation = self.activation
        
        ## INFERENCE ##
        self.inference_net = tf.keras.Sequential( [
            tf.keras.layers.Input(shape=(feature_dim,)),
            models.AEFIT3.NaNDense(feature_dim, activation=activation),
            tf.keras.layers.Dense(latent_dim * 200 * scale, activation=activation),
            tf.keras.layers.Dropout(dprate),
            tf.keras.layers.Dense(latent_dim * 200 * scale, activation=activation),
            tf.keras.layers.Dropout(dprate),
            tf.keras.layers.Dense(latent_dim * 100 * scale, activation=activation),            
            tf.keras.layers.Dropout(dprate),
            tf.keras.layers.Dense(latent_dim * 100 * scale, activation=activation),
            tf.keras.layers.Dense(6 * latent_dim),
            tf.keras.layers.Dense(2 * latent_dim),
            ] )
        


        ## GENERATION ##
        self.generative_net = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(latent_dim,)),
            tf.keras.layers.Dense(units=latent_dim),
            tf.keras.layers.Dense(latent_dim * 100 * scale, activation=activation),            
            tf.keras.layers.Dropout(dprate),
            tf.keras.layers.Dense(latent_dim * 100 * scale, activation=activation),            
            tf.keras.layers.Dropout(dprate),
            tf.keras.layers.Dense(latent_dim * 200 * scale, activation=activation),
            tf.keras.layers.Dropout(dprate),
            tf.keras.layers.Dense(latent_dim * 200 * scale, activation=activation),
            tf.keras.layers.Dense(units=feature_dim),
        ], name = 'generative_net')
        
        
        self.inference_net.build()
        self.generative_net.build()

        # self.inputs = self.inference_net.inputs
        # self.outputs = self.generative_net.outputs
        self._sce = 0.
        self._kld = 0.
        self._akl = 0.
        self._v_mea = 0.
        self._v_std = 0.
        # Compile the model
        self.compile(  
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
            loss = self.vae3_loss,
            metrics = ['accuracy', self.sce, self.akl, self.kld, self.v_mea, self.v_std]
        )
        # self.build(input_shape=self.inference_net.input_shape)



In [4]:
qsh = Htls.QSH_Dataset()
import os
file = ipysh.abs_builddir+'/te_db_r15.npy'
if os.path.isfile(file):
    qsh.load(file)
else:
    qsh.load(ipysh.abs_builddir+'/te_db_1.npy')    
    qsh.rebalance_prel(15)
    qsh.save(ipysh.abs_builddir+'/te_db_r15.npy')
    

qsh.shuffle()
qsh.clean_up_poorcurves(5)
qsh.dim = 15
qsh.set_null(np.nan)
qsh.set_normal_positive()
qsh.unbias_mean(0.5, 'te')
qsh.set_normal_positive()


qsh.set_null(np.nan)
qsh.clip_values(0.1,0.6)
qsh.set_normal_positive()

print("QSH rebalanced 15 points size: ", len(qsh))

QSH rebalanced 15 points size:  61519


# Normal AEFIT3

In [5]:
m = models.AEFIT3.AEFIT3(latent_dim=2, feature_dim=30,  dprate=0., scale=1, beta=1.)

AEFIT3 ready:


In [6]:
p = Hplt.LSPlotBokeh()
p.set_model(m)
p.set_data(qsh, counts=1000)
p.plot(notebook_url='http://172.17.0.2:8888')

W0819 08:47:17.710712 140369005573952 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:504: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
W0819 08:47:17.735196 140369005573952 deprecation.py:323] From /home/andrea

Loading BokehJS ...

In [7]:
models.base.train_thread(m, qsh, batch=200, epoch=4, learning_rate=1e-3, callbacks=[]).control_panel()

Epoch 1/4


W0819 08:47:23.362948 140365638137600 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:182: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


307/307 [==============================] - 11s 36ms/step - loss: 18.4651 - accuracy: 0.8474 - sce: 18.6812 - akl: 10.0788 - kld: 5.8130 - v_mea: 1.7516 - v_std: 0.7109
Epoch 2/4
307/307 [==============================] - 9s 29ms/step - loss: 18.3843 - accuracy: 0.9894 - sce: 18.3965 - akl: 11.3496 - kld: 7.2777 - v_mea: 1.3880 - v_std: 0.1875
Epoch 3/4
307/307 [==============================] - 9s 29ms/step - loss: 18.3615 - accuracy: 0.9900 - sce: 18.3730 - akl: 13.8004 - kld: 10.2974 - v_mea: 0.2526 - v_std: 0.0303
Epoch 4/4
307/307 [==============================] - 9s 31ms/step - loss: 18.3532 - accuracy: 0.9914 - sce: 18.3593 - akl: 15.2364 - kld: 11.8782 - v_mea: -0.0464 - v_std: 0.0142


# Add layer

Added one layer with 12 nodes before latent space reduction

In [8]:
a = AEFIT3a(latent_dim=2, feature_dim=30,  dprate=0., scale=1, beta=1.)

AEFIT3 ready:
AEFIT3a ready:


In [9]:
p = Hplt.LSPlotBokeh()
p.set_model(a)
p.set_data(qsh, counts=1000)
p.plot(notebook_url='http://172.17.0.2:8888')

Loading BokehJS ...

In [10]:
models.base.train_thread(a, qsh, batch=200, epoch=4, learning_rate=1e-3, callbacks=[]).control_panel()

Epoch 1/4
307/307 [==============================] - 11s 35ms/step - loss: 18.4551 - accuracy: 0.8564 - sce: 18.6495 - akl: 21.8847 - kld: 10.6930 - v_mea: 0.7111 - v_std: 0.4164
Epoch 2/4
307/307 [==============================] - 9s 29ms/step - loss: 18.3738 - accuracy: 0.9894 - sce: 18.3875 - akl: 32.1114 - kld: 15.7695 - v_mea: 0.4485 - v_std: 0.0645
Epoch 3/4
307/307 [==============================] - 9s 29ms/step - loss: 18.3565 - accuracy: 0.9901 - sce: 18.3645 - akl: 21.0890 - kld: 11.7985 - v_mea: 0.2646 - v_std: 0.0608
Epoch 4/4
307/307 [==============================] - 9s 30ms/step - loss: 18.3530 - accuracy: 0.9914 - sce: 18.3588 - akl: 22.4447 - kld: 13.1975 - v_mea: 0.2198 - v_std: 0.0308
